<a href="https://colab.research.google.com/github/computerphotophytoart/Oyster-Mussel-Classifier/blob/main/OysterMusselClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastai
from fastai.vision.all import * #This imports fastai library, -Uqq is used to silence output msgs

In [ ]:
foodpath = untar_data(URLs.FOOD) #unpacks food dataset

In [ ]:
len(get_image_files(foodpath)) #show value of foods in dataset (should be 101000)

In [ ]:
pd.read_json('/root/.fastai/data/food-101/test.json') #show food dataset in readable format

In [ ]:
labelA = 'oysters' #choosing the foods we want to classify
labelB = 'mussels'

In [ ]:
for img in get_image_files(foodpath):
  #Check if labelA is in file name
  if labelA in str(img):
    img.rename(f"{img.parent}/{labelA}-{img.name}")
  elif labelB in str(img):
    img.rename(f"{img.parent}/{labelB}-{img.name}")
  else: os.remove(img)
#If labelA or labelB aren't found, remove the files
len(get_image_files(foodpath)) #get number of images in directory

**Model Training**

In [ ]:
def GetLabel(fileName):
  return fileName.split('-')[0]

GetLabel("oysters-987252.jpg") #takes file name of an image as input. if the file name is 'oysters-987252.jpg', it extracts the label 'oysters' and returns it."

In [ ]:
dls = ImageDataLoaders.from_name_func( #creates data loaders
    foodpath, get_image_files(foodpath), valid_pct=0.2, seed=420, #retrieves files
    label_func=GetLabel, item_tfms=Resize(32)) #resizes images to 32. increase for higher quality images

dls.train.show_batch() #display images from training set

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate, pretrained=True) #creates convolutional neural network with resnet34.
#you can change resnet34 to resnet50 for a more modern architecture but unfortunately Colab has resource limitations
learn.fine_tune(epochs=1) #higher epochs means model cycles through dataset more times (1 = 1 cycle). Keep between 1-3

**Model Testing**

In [ ]:
from google.colab import files
uploader = files.upload() #allow file uploading

for img in uploader.items():
  uploadedImg = img[0]

img = PILImage.create(uploadedImg)
img.show()

label,_,probs = learn.predict(img)

print(f"This is a {label}.") #prints prediction
print(f"{labelA} {probs[1].item():.6f}")
print(f"{labelB} {probs[0].item():.6f}") #prints prediction with probability from 0 - 1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for i in range(0,10):
  #load random image
  randomIndex = random.randint(0, len(get_image_files(foodpath))-1)
  img = mpimg.imread(get_image_files(foodpath)[randomIndex])
  #put into model
  label,_,probs = learn.predict(img)

  #create figure with matplotlib
  fig = plt.figure()
  ax = fig.add_subplot()
  imgplot = plt.imshow(img) #add image to plot
  ax.set_title(label) #set title to label prediction

  #remove numbers from axis
  plt.gca().axes.get_yaxis().set_visible(False)
  plt.gca().axes.get_xaxis().set_visible(False)

In [ ]:
#confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(6)

**Download Model**

In [ ]:
learn.export() #export the model into a file

In [ ]:
modelPath = get_files(foodpath, '.pkl')[0] #get pkl file
modelPath

In [ ]:
learn_inf = load_learner(modelPath) #load model from file
learn_inf.predict(mpimg.imread(get_image_files(foodpath)[0])) #use model to read label and predict image

In [ ]:
learn_inf.dls.vocab #return which labels model can classify

In [ ]:
from google.colab import files
files.download(modelPath) #download model